# Práctica 3: Fundamentos CNN

En primer lugar vamos a tratar de resolver el problema de clasificación con la base de datos CIFAR 10 que no pudimos solventar en la práctica anterior empleando redes neuronales totalmente conectadas.

Para el desarrollo de esta práctica vamos a activar el uso de GPU. Para ellos accede a: 
Entorno de ejecución -> Cambiar tipo de entorno de ejecución -> Acelerador por hardware -> GPU -> Guardar

In [ ]:
# Importamos la base de datos
from tensorflow.keras.datasets import cifar10

(X_train, y_train), (X_testval, y_testval) = cifar10.load_data()

print(X_train.shape, y_train.shape)
print(X_testval.shape, y_testval.shape)

print('Valor mínimo imágenes: ', X_train.min())
print('Valor máximo imágenes: ', X_train.max())

In [ ]:
import matplotlib.pyplot as plt

# Vector de los nombres de las clases definidas en CIFAR
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 
           'horse', "ship", "truck"]

# Mostramos 9 imágenes
plt.rcParams['figure.figsize'] = (10, 10)
for i in range(9):
  plt.subplot(3, 3, i+1)
  plt.imshow(X_train[i])
  plt.title(f'{classes[int(y_train[i])]}')

In [3]:
# Convertimos rango imágenes a 0-1
X_train = X_train.astype('float32')
X_testval = X_testval.astype('float32')
X_train /= 255
X_testval /= 255

In [ ]:
from sklearn.preprocessing import OneHotEncoder
# Convertimos etiquetas a codificación one-hot
from tensorflow.keras.utils import to_categorical
num_clases = len(np.unique(y_train))
y_train_cod = to_categorical(y_train, num_clases)
y_testval_cod = to_categorical(y_testval, num_clases)
print("Tamaño etiquetas entrenamiento: ", y_train_cod.shape)
print("Tamaño etiquetas validación/test: ", y_testval_cod.shape)

In [ ]:
# Dividimos conjunto de datos de validación/test en 2 subconjuntos
from sklearn.model_selection import train_test_split
X_val, X_test, y_val, y_test = train_test_split(X_testval, y_testval_cod,
                                                test_size=0.5)

print("Muestras validación: ", X_val.shape)
print("Salida validación: ", y_val.shape)
print("Muestras test: ", X_test.shape)
print("Salida test: ", y_test.shape)

In [ ]:
# Definimos la arquitectura
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense

# Extracción caracteríisticas
input_layer = Input(shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3]))
conv1 = Conv2D(filters=8, kernel_size=(3,3), activation='relu')(input_layer)
max_pool1 = MaxPooling2D(pool_size=(2,2), strides=(2,2))(conv1)

conv2 = Conv2D(filters=16, kernel_size=(3,3), activation='relu')(max_pool1)
max_pool2 = MaxPooling2D(pool_size=(2,2), strides=(2,2))(conv2)

conv3 = Conv2D(filters=32, kernel_size=(3,3), activation='relu')(max_pool2)
max_pool3 = MaxPooling2D(pool_size=(2,2), strides=(2,2))(conv3)

# Clasificación
flatten_layer = Flatten()(max_pool3) 
hidden_layer = Dense(128, activation='relu')(flatten_layer)
output_layer = Dense(10, activation='softmax')(hidden_layer) # 10 salidas

model = Model(inputs=input_layer, outputs=output_layer)

model.summary()

In [7]:
# Compilamos el modelo
model.compile(loss="categorical_crossentropy", optimizer="adam",
              metrics=["accuracy"])

In [ ]:
# Entenamos el modelo
history = model.fit(X_train, y_train_cod, epochs=20, batch_size=128,
                    validation_data=(X_val, y_val))

In [ ]:
# Visualizamos la precisión
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Precisión modelo')
plt.ylabel('Precisión')
plt.xlabel('Época')
plt.ylim(0,1)
plt.legend(['Entrenamiento', 'Validación'], loc="lower right")
plt.show()

In [ ]:
# Evaluamos modelos
metrics = model.evaluate(X_test, y_test, verbose=0)
print('Precisión test: ', metrics[1])

In [ ]:
import numpy as np

# Obtenemos predicciones 
prediccion = model.predict(X_test)
# Cogemos la clase con mayor probabilidad
prediccion = np.argmax(prediccion, axis=1)

# y_test lo tenemos en codificación OneHot, por lo que lo convertimos a clases
y_test_clases = np.argmax(y_test, axis=1)

# Buscamos los índces de las imágenes corretamente e incorrectamente clasificadas
correct_index = np.nonzero(prediccion == y_test_clases)[0]
incorrect_index = np.nonzero(prediccion != y_test_clases)[0]

# Mostramos 9 imágenes correctamente clasifcadas
plt.rcParams['figure.figsize'] = (10, 10)
for i, correct in enumerate(correct_index[:9]):
  plt.subplot(3, 3, i+1)
  plt.imshow(X_test[correct])
  plt.title(f'R: {classes[int(y_test_clases[correct])]}, P: {classes[int(prediccion[correct])]}')

In [ ]:
# Mostramos 9 imágenes incorrectamente clasificadas
for i, incorrect in enumerate(incorrect_index[:9]):
  plt.subplot(3, 3, i+1)
  plt.imshow(X_test[incorrect])
  plt.title(f'R: {classes[int(y_test_clases[incorrect])]}, P: {classes[int(prediccion[incorrect])]}')

# Ejercicio 1: Entramiento durante más épocas
Vamos a entrenar el modelo durante 50 épocas

NOTA: Para cada ejercicio genera una modelo diferente (con distinto nombre)

# Ejercicio 2: Doblamos el número de filtros en cada capa convolucional

Vamos a aumentar la complejidad de la arquitectura. Primero vamos a doblar el número de filtros de cada capa convolucional:
- conv1: 8 -> 16
- conv2: 16 -> 32
- conv3: 32 -> 64

Para estas pruebas vuelve a poner el número de épocas a 20 para que los experimentos vayan más rápido.

# Ejercicio 3. Aumentamos el número de capas convolucionales.
Tomando como partida el modelo inicial, donde ahora tenemos una capa convolucional añade otra justo antes con el mismo número de filtros y padding "same" (tamaño de salida igual al tamaño de entrada).

# Ejercicio 4: Aumentamos el tamaño de batch del modelo inicial
Vamos a cambiar el tamaño de batch a 250.

Peores resultados con mayor tamaño de batch!!

Puede parecer intuitivo que si ampliamos el tamaño de batch los resultados serán mejores ya que el modelo "ve" más imágenes de una. Pero esto no siempre es así. Hay que seleccionar el tamaño de batch con cuidado.

# Aumento de datos

In [40]:
# Definimos generador de datos
from keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(
                                    rotation_range=5, 
                                    horizontal_flip=True,
                                    zoom_range=.3)

train_generator.fit(X_train)

In [ ]:
# Visualizamos imágenes aumentadas
augmented_images, _ = next( train_generator.flow(X_train, y_train_cod, batch_size=9))
plt.rcParams['figure.figsize'] = (10, 10)
for i in range(9):
  plt.subplot(3, 3, i+1)
  plt.imshow(augmented_images[i, :, :, :])

In [ ]:
# Definimos arquitectura
input_layer = Input(shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3]))
conv1 = Conv2D(filters=8, kernel_size=(3,3), activation='relu')(input_layer)
max_pool1 = MaxPooling2D(pool_size=(2,2), strides=(2,2))(conv1)

conv2 = Conv2D(filters=16, kernel_size=(3,3), activation='relu')(max_pool1)
max_pool2 = MaxPooling2D(pool_size=(2,2), strides=(2,2))(conv2)

conv3 = Conv2D(filters=32, kernel_size=(3,3), activation='relu')(max_pool2)
max_pool3 = MaxPooling2D(pool_size=(2,2), strides=(2,2))(conv3)

flatten_layer = Flatten()(max_pool3) 
hidden_layer = Dense(128, activation='relu')(flatten_layer)
output_layer = Dense(10, activation='softmax')(hidden_layer) # 10 salidas

model_6 = Model(inputs=input_layer, outputs=output_layer)

# Compilamos el modelo
model_6.compile(loss="categorical_crossentropy", optimizer="adam",
                metrics=["accuracy"])
# Entrenamos

history_6 = model_6.fit_generator(train_generator.flow(X_train, y_train_cod, batch_size=128),
                                            epochs=20,
                                            steps_per_epoch=X_train.shape[0] // 128,
                                            validation_data=(X_val, y_val))



In [ ]:
# Visualizamos la precisión
plt.plot(history_6.history['accuracy'])
plt.plot(history_6.history['val_accuracy'])
plt.title('Precisión modelo')
plt.ylabel('Precisión')
plt.xlabel('Época')
plt.ylim(0,1)
plt.legend(['Entrenamiento', 'Validación'], loc="lower right")
plt.show()

In [ ]:
# Comparamos métricas entrenamento/validación con modelo inicial
print('Precisión modelo inicial: ')
print(' - Entrenamiento: ', history.history['accuracy'][-1])
print(' - Validación: ', history.history['val_accuracy'][-1])
print('Precisión modelo con aumento de datos: ')
print(' - Entrenamiento: ', history_6.history['accuracy'][-1])
print(' - Validación: ', history_6.history['val_accuracy'][-1])

In [ ]:
# Comparamos las métricas en validación con el modelo inicia,
plt.plot(history.history['val_accuracy'])
plt.plot(history_6.history['val_accuracy'])
plt.title('Precisión modelo (Validación)')
plt.ylabel('Precisión')
plt.xlabel('Época')
plt.ylim(0,1)
plt.legend(['Modelo inicial', 'Modelo con aumento de datos'], loc="lower right")
plt.show()

In [ ]:
# Evaluamos sobre el conjunto de test y comparamos con modelo inicial
metrics_6 = model_6.evaluate(X_test, y_test, verbose=0)
print('Precisión modelo inicial (test): ', metrics[1])
print('Precisión modelo con aumento de datos (test): ', metrics_6[1])

Tras aplicar aumento de datos observamos una pequeña mejora de la precisión.

Investiga el método de DataGenerator y prueba con más transformaciones. Incluso es posible aplicar aumento de datos al set de validación!

# Transferencia de conocimiento

En este apartado vamos a aplicar tranferencia de conocimineto empleando un modelo pre-entrenado en la base de datos de Imagenet. Concretamente emplearemos el modelo ResNet50.

In [ ]:
from tensorflow.keras.applications.resnet50 import preprocess_input
import numpy as np

# Volvemos al rango inicial de la imágenes [0-255] ya que se van a preprocesar de acuerdo a las necesidades de la red.
X_train *= 255
X_val *= 255
X_test *= 255
print("Rango imágenes: [", X_train.min(), ', ', X_train.max(),']')

# Preprocesamos los datos para que se ajusten a las necesidades del modelo selecccionado
X_train_prep = preprocess_input(X_train)
X_val_prep = preprocess_input(X_val)
X_test_prep = preprocess_input(X_test)

print('Tamaño imágenes entrenamiento: ', X_train_prep.shape)
print('Valor mínimo imágenes entrenamiento: ', X_train_prep.min())
print('Valor máximo imágenes entrenamiento: ', X_train_prep.max())

In [ ]:
from keras.applications import ResNet50

# Cogemos modelo base a utilizar (ResNet50).
# Con include_top=False le estamos indicando que no queremos incluir la última capa (FC clasificación final)
# Con weights='imagenet' le estamos indicando que queremos inicializar los pesos con los que se obtuvieron en el entrenamiento con la BBDD de Imagenet.
base_model = ResNet50(include_top=False,weights='imagenet', input_shape=(32,32,3))
base_model.summary()

No vamos a usar todas las capas de ResNet ya que podemos ver que muchos de los bloques acaban teniendo un tamaño de imagen de 1x1. Esto es debido a que las imágenes de CIFAR son mucho más pequeñas (32x32) que las de ImageNet (224x224).

Vamos a coger hasta la capa "conv3_block4_out", que tiene un tamaño de imagen de 2x2 y vamos a congelar todas las capas. Es decir, durante el entrenamiento, los pesos de estas capas no se van a alterar, vamos a dejar los obtenidos en el entrenamiento con la base de datos de ImageNet. 
A estas capas les vamos a incluir, a la salida, una fase de clasificación, que será aquello que entrenaremos.

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, BatchNormalization, GlobalAveragePooling2D, Dropout

# Congelamos todas las capas del clasificador VGG

# Cogemos la salida de la capa "conv3_block4_out"
#x = base_model.output
x = base_model.get_layer('conv3_block4_out').output

# Añadimos capas de clasificación
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output_layer = Dense(10, activation='softmax')(x)
model_resnet = Model(inputs=base_model.input, outputs=output_layer)

for layer in base_model.layers:
     layer.trainable = False

model_resnet.summary()



Fijaros en el número de parámetros entrenables!! Todos los de ResNet no se van a alterar, únicamente los que hemos añadido con la capa totalmente conectada.

In [51]:
# Compilamos el modelo
model_resnet.compile(loss="categorical_crossentropy", optimizer="adam",
                  metrics=["accuracy"])

In [ ]:
# Entenamos el modelo
history_resnet = model_resnet.fit(X_train_prep, y_train_cod, epochs=20, 
                                  batch_size=128,
                                  validation_data=(X_val_prep, y_val))

In [ ]:
# Visualizamos la precisión
import matplotlib.pyplot as plt
plt.plot(history_resnet.history['accuracy'])
plt.plot(history_resnet.history['val_accuracy'])
plt.title('Precisión modelo')
plt.ylabel('Precisión')
plt.xlabel('Época')
plt.legend(['Entrenamiento', 'Validación'], loc="upper left")
plt.show()

In [ ]:
# Comparativa con modelo inicial
plt.plot(history.history['val_accuracy'])
plt.plot(history_resnet.history['val_accuracy'])
plt.title('Precisión modelo (validación')
plt.ylabel('Precisión')
plt.xlabel('Época')
plt.legend(['CNN propia', 'Transfer learning (ResNet)'], loc="lower right")
plt.show()

In [ ]:
# Evaluamos sobre conjunto de test
metrics_resnet = model_resnet.evaluate(X_test_prep, y_test, verbose=0)
print("Precision test CNN propia: ", metrics[1])
print("Precision test transferencia conocimiento: ", metrics_resnet[1])

Podemos observar un aumento en la precisión.

Tras haber realizado una transferencia de conocimiento congelando las capas del modelo base, se puede hace un ajuste fino del modelo adaptando los pesos de toda la red empleando una tasa de aprendizaje baja durante pocas épocas (para evitar el sobreajuste).

In [ ]:
from tensorflow.keras.optimizers import Adam
# Descongelamos las capas del modelo base
for layer in base_model.layers:
	layer.trainable = True

# Compilamos el modelo con una tasa de aprendizaje más baja
model_resnet.compile(optimizer=Adam(1e-5),
              loss= 'categorical_crossentropy',
              metrics=['accuracy'])

# Entrenamos modelo durante unas pocas épocas.
history_resnet_fine = model_resnet.fit(X_train_prep, y_train_cod, epochs=10, 
                                       batch_size=128,
                                       validation_data=(X_val_prep, y_val))

In [ ]:
# Evaluamos sobre conjunto de test
metrics_resnet_2 = model_resnet.evaluate(X_test_prep, y_test, verbose=0)
print("Precision test CNN propia: ", metrics[1])
print("Precision test transferencia conocimiento: ", metrics_resnet_2[1])

In [ ]:
# Obtenemos predicciones 
prediccion = model_resnet.predict(X_test_prep)
# Cogemos la clase con mayor probabilidad
prediccion = np.argmax(prediccion, axis=1)

# y_test lo tenemos en codificación OneHot, por lo que lo convertimos a clases
y_test_clases = np.argmax(y_test, axis=1)

# Buscamos los índces de las imágenes corretamente e incorrectamente clasificadas
correct_index = np.nonzero(prediccion == y_test_clases)[0]
incorrect_index = np.nonzero(prediccion != y_test_clases)[0]

# Mostramos 9 imágenes correctamente clasificadas
plt.rcParams['figure.figsize'] = (10, 10)
for i, correct in enumerate(correct_index[:9]):
  # Convertimos imagen al rango [0, 1] para visualizarla
  image = X_test[correct]
  for j in range(3):
    image_ch = image[:, :, j]
    image[:, :, j] = (image_ch - image_ch.min()) / (image_ch.max() - image_ch.min())
  plt.subplot(3, 3, i+1)
  plt.imshow(image)
  plt.title(f'R: {classes[int(y_test_clases[correct])]}, P: {classes[int(prediccion[correct])]}')

In [ ]:
# Mostramos 9 imágenes incorrectamente clasificadas
plt.rcParams['figure.figsize'] = (10, 10)
for i, incorrect in enumerate(incorrect_index[:9]):
  # Convertimos imagen al rango [0, 1] para visualizarla
  image = X_test[incorrect]
  for j in range(3):
    image_ch = image[:, :, j]
    image[:, :, j] = (image_ch - image_ch.min()) / (image_ch.max() - image_ch.min())
  plt.subplot(3, 3, i+1)
  plt.imshow(image)
  plt.title(f'R: {classes[int(y_test_clases[incorrect])]}, P: {classes[int(prediccion[incorrect])]}')